#### this code does not include pine needle / cones measurements !! 
- probs for this, just want mean length for each site
- are adding in cone / needle length


** need to re-write:
- remove nearest neighbour
- add in litter, aspect
- needle will be in a seperate csv file

In [16]:
import csv
import numpy as np
import pandas as pd

################ bits to edit before running #########################
tree_csv_data = 'tree_data_1.csv'
collated_transect_csv = 'transect_1.csv'
transect_file_name = 'practice-format.csv'

# tree site area (m2)
site_area = 100

# number sites on transect?
n_sites = 3

#######################################################################
# make sure csv index is aligned + got shape of df
transect_data = pd.read_csv(transect_file_name)
transect_data = transect_data.reset_index() 
transect_rows,transect_columns = transect_data.shape

# max index value for row --> transect_rows-2 (as column names = -1, then index start on 0)

# get rid of empty values
transect_data = transect_data.replace({np.nan: None})

site_names = []
variables = ['Time','Pressure','Altitude','biomass_density','nearest_neighbour', 'needle', 'cone','wind_speed','wind_direction','RH','Temp','Lat','Long']
tree_variables = ['diameter','basal_area','biomass']

# make list of site names
for i in range(n_sites):
    site_names.append('site-'+str(i+1))
     
# create empty dictionary of tree data for transect sites
site_section_data = {}
for name in site_names:
    site_section_data[name] = pd.DataFrame()

# loop through rows + make list of empty rows using 1st column (tree_circumference) - use to split up sites
# i.e. where there is a gap in tree circumference values, will return the index of empty cell 
## 

## works !
empty_row = []
for index, row in transect_data.iterrows():
    if transect_data.at[index,'tree_circumference'] == None:
        empty_row.append(index)

# make 2 list of start + end index of sites ##
# works !!
site_row_start = [0]
site_row_end = []
for i in range(n_sites-1):
    #print(n_sites-1)
    site_row_start.append(int(empty_row[i]+1))
    site_row_end.append(int(empty_row[i]))
    #print(i)
    if i == n_sites-2:
        site_row_end.append(transect_rows)    
    
    
## section up the sites data from main csv file --> dictionary of sites, each with dictionary of data 
## works !
for i in range(len(site_names)):
    site_section_data[site_names[i]] = pd.DataFrame()
    site_section_data[site_names[i]] = transect_data.iloc[site_row_start[i]:site_row_end[i],:]
    
    ## access particular site:     # site_section_data['site-1']
    ## access particular sub-set of data:    # site_section_data['site-1']['tree_circumference']

## writes 'tree_circumference' data into seperate csv file
# works !
empty = pd.DataFrame([0])
for i in range(len(site_names)):
    if i == 0:
        site_section_data['site-'+str(i+1)]['tree_circumference'].to_csv(tree_csv_data,index=False, mode='w')
    else:
        empty.to_csv(tree_csv_data,index=False, header=False, mode='a')
        site_section_data['site-'+str(i+1)]['tree_circumference'].to_csv(tree_csv_data,index=False,header=False, mode='a')

############# 
## edit + manipulate tree data --> all calculations
all_tree_data = pd.read_csv(tree_csv_data)

t_diameter = []
t_basal_area =[]
t_biomass = []

"""# !!!!!!!  tree data processing !!!!!!!!!                                #### check units are alright ########"""
for i in range(len(all_tree_data)):
    cir = all_tree_data['tree_circumference'][i]
    di = cir/np.pi
    t_diameter.append(di)
    ba = np.pi*(di/2)**2
    t_basal_area.append(ba)
    ## need to add biomass calculations??      !!! ######################################################## !!!!!
    ## currently pretend ones #########################################
    bio = ba * 10
    t_biomass.append(bio)
    #########################################################################

"""treessss                                         #######################################################################################"""


# adding new column to all_tree_data ##
## works !!! 
all_tree_data[tree_variables[0]] = t_diameter
all_tree_data[tree_variables[1]] = t_basal_area
all_tree_data[tree_variables[2]] = t_biomass

all_tree_data = all_tree_data.replace({0: None})

##  put tree data into csv file ################## so now all tree numbers are in 1 place #################
all_tree_data.to_csv(tree_csv_data,index=False, mode='w')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  now we want to compile the other bits to 1 csv  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
collated_data = pd.DataFrame(index = site_names)

all_tree_data = pd.read_csv(tree_csv_data)

# need to make individual lists for mean + group variables
tim = []
pre = []
alt = []

ws = []
wsl = []
wd = []
wdl = []
rh = []
rhl = []
temp = []
templ = []
ne = []
nel = []
co = []
col = []

lat = []
lon = []

nn = []
nnl = []
bd = []
bdl = []

print(range(n_sites))

for j in range(n_sites):
    # add list of values to 
    tim.append(site_section_data[site_names[j]]['Time'][site_row_start[j]])
    pre.append(site_section_data[site_names[j]]['Pressure'][site_row_start[j]])
    alt.append(site_section_data[site_names[j]]['altitude'][site_row_start[j]])
    lat.append(site_section_data[site_names[j]]['Lat'][site_row_start[j]])
    lon.append(site_section_data[site_names[j]]['Long'][site_row_start[j]])

    for t in range(site_row_end[j]-site_row_start[j]):
        nn.append(site_section_data[site_names[j]]['nearest_neighbour'][site_row_start[j]+t])
        bd.append(all_tree_data['biomass'][site_row_start[j]+t])
        
    nn = np.mean(nn)
    bd = np.sum(bd)

############################## calculate nearest neighbour for each site:       #########################################
    nnl.append((nn)/(0.5*np.sqrt(site_area/(site_row_end[j]-site_row_start[j]))))
    ############################## calculate biomass for each site:      #########################################
    bdl.append(bd/site_area)

    nn = []
    bd = []

    for i in range(3):
        # this is the loop to get mean of 3 values
        ws.append(site_section_data[site_names[j]]['Wind_speed'][site_row_start[j]+i])
        wd.append(site_section_data[site_names[j]]['Wind_direction'][site_row_start[j]+i])
        rh.append(site_section_data[site_names[j]]['RH'][site_row_start[j]+i])
        temp.append(site_section_data[site_names[j]]['Temperature'][site_row_start[j]+i])
    wsl.append(np.mean(ws))
    wdl.append(np.mean(wd))
    rhl.append(np.mean(rh))
    templ.append(np.mean(temp))
    ws = []
    wd = []
    rh = []
    temp = []

    for p in range(site_row_end[j]-site_row_start[j]):
        if site_section_data[site_names[j]]['needle'][site_row_start[j]+p] != None:
            ne.append(site_section_data[site_names[j]]['needle'][site_row_start[j]+p])
        if site_section_data[site_names[j]]['cone'][site_row_start[j]+p] != None:
            co.append(site_section_data[site_names[j]]['cone'][site_row_start[j]+p])
    #nel.append(np.mean(np.nonzero(ne)))
    #col.append(np.mean(np.nonzero(co)))
    nel.append(np.mean(ne))
    col.append(np.mean(co))
    print(ne)
    print(co)
    ne =[]
    co = []

collated_data.insert(0,'Time',tim,allow_duplicates = True)
collated_data.insert(1,'Pressure',pre,allow_duplicates = True)
collated_data.insert(2,'Altitude',alt,allow_duplicates = True)

collated_data.insert(3,'biomass_density',bdl,allow_duplicates = True)
collated_data.insert(4,'nearest_neighbour',nnl,allow_duplicates = True)
collated_data.insert(5,'needle',nel,allow_duplicates = True)
collated_data.insert(6,'cone',col,allow_duplicates = True)

collated_data.insert(7,'wind_speed',wsl,allow_duplicates = True)
collated_data.insert(8,'wind_direction',wdl,allow_duplicates = True)
collated_data.insert(9,'RH',rhl,allow_duplicates = True)
collated_data.insert(10,'Temp',templ,allow_duplicates = True)
collated_data.insert(11,'Lat',lat,allow_duplicates = True)
collated_data.insert(12,'Long',lon,allow_duplicates = True)

collated_data.to_csv(collated_transect_csv, mode='w')

range(0, 3)
[20.0, 21.0, 20.0, 22.0, 21.0, 20.0]
[18.0, 19.0, 18.0, 17.0, 19.0, 20.0]
[25.0, 24.0, 26.0, 23.0]
[22.0, 23.0, 21.0, 20.0]
[32.0, 30.0, 31.0, 29.0, 28.0, 30.0]
[24.0, 25.0, 23.0, 21.0, 26.0, 23.0]


- need to update units
- need to do biomass calculation (not over specific area though) - mean tree biomass for each site + want it all onto 1 csv, actually, stil have the biomass seperate
- still need to do needle numbers - min + max + mean?

In [27]:
import csv
import numpy as np
import pandas as pd

################ bits to edit before running #########################
tree_csv_data = 'tree_data_1.csv'
collated_transect_csv = 'transect_1_processed.csv'
transect_file_name = 'transect_1.csv'

# tree site area (m2)
site_area = 100

# number sites on transect?
n_sites = 9

### idk why but the empty row doesn't want to work, so done it manually

#######################################################################
# make sure csv index is aligned + got shape of df
transect_data = pd.read_csv(transect_file_name)
transect_data = transect_data.reset_index() 
transect_rows,transect_columns = transect_data.shape

#print(transect_rows)
#print(transect_columns)

# max index value for row --> transect_rows-2 (as column names = -1, then index start on 0)

# get rid of empty values
transect_data = transect_data.replace({np.nan: None})

site_names = []
variables = ['Time','Pressure','Altitude','biomass_density','needle', 'cone','wind_speed','wind_direction','RH','Temp','Lat','Long', 'aspect','litter']
tree_variables = ['diameter','basal_area','biomass']

# make list of site names
for i in range(n_sites):
    site_names.append('site-'+str(i+1))
     
# create empty dictionary of tree data for transect sites
site_section_data = {}
for name in site_names:
    site_section_data[name] = pd.DataFrame()

# loop through rows + make list of empty rows using 1st column (tree_circumference) - use to split up sites
# i.e. where there is a gap in tree circumference values, will return the index of empty cell 
## 
### with this. may be more cone measurements --> so need to take this into account, and empty values
## works !
empty_row_tr = []
empty_row_con = []
empty_row_needle =[]
empty_row = []

# this is choosing the largest sections of data, from tree circ // cone length // needle length
for index, row in transect_data.iterrows():
    if transect_data.at[index,'tree_circumference'] == None and transect_data.at[index-1,'tree_circumference'] != None:
        #empty_row_tr.append(index+1)
        empty_row_tr.append(index)        
        
    if transect_data.at[index,'cone'] == None and transect_data.at[index-1,'cone'] != None:
        #empty_row_con.append(index+1)
        empty_row_con.append(index)

    if transect_data.at[index, 'needle_l'] == None and transect_data.at[index-1,'needle_l'] != None:
        empty_row_needle.append(index)


for i in range(len(empty_row_tr)):
    tr = empty_row_tr[i]
    co = empty_row_con[i]
    ne = empty_row_needle[i]

    if (tr >= co) and (tr >= ne):
        empty_row.append(tr)
    elif (co >= tr) and (co >= ne):
        empty_row.append(co)  
    else:
        empty_row.append(ne)


"""
    if tr == co and tr == ne:
        empty_row.append(tr)
    if tr > co and tr >= ne:
        empty_row.append(tr)
    if ne > tr and ne > co:
        empty_row.append(ne)
    else:
        empty_row.append(co)
"""

#empty_row = [5,]
print(empty_row)

# make 2 list of start + end index of sites ##
# works !!
site_row_start = [0]
site_row_end = []
for i in range(n_sites-1):
    #print(n_sites-1)
    site_row_start.append(int(empty_row[i]+1))
    site_row_end.append(int(empty_row[i]))
    #print(i)
    if i == n_sites-2:
        site_row_end.append(transect_rows)    
    
    
## section up the sites data from main csv file --> dictionary of sites, each with dictionary of data 
## works !
for i in range(len(site_names)):
    site_section_data[site_names[i]] = pd.DataFrame()
    site_section_data[site_names[i]] = transect_data.iloc[site_row_start[i]:site_row_end[i],:]
    
    ## access particular site:     # site_section_data['site-1']
    ## access particular sub-set of data:    # site_section_data['site-1']['tree_circumference']

## writes 'tree_circumference' data into seperate csv file
# works !
empty = pd.DataFrame([0])
for i in range(len(site_names)):
    if i == 0:
        site_section_data['site-'+str(i+1)]['tree_circumference'].to_csv(tree_csv_data,index=False, mode='w')
    else:
        empty.to_csv(tree_csv_data,index=False, header=False, mode='a')
        site_section_data['site-'+str(i+1)]['tree_circumference'].to_csv(tree_csv_data,index=False,header=False, mode='a')

############# 
## edit + manipulate tree data --> all calculations
all_tree_data = pd.read_csv(tree_csv_data)

t_diameter = []
t_basal_area =[]
t_biomass = []

# !!!!!!!  tree data processing !!!!!!!!!                                #### check units are alright ########""""""
for i in range(len(all_tree_data)):
    cir = all_tree_data['tree_circumference'][i]
    di = (cir/np.pi) *100 # diameter is in cm
    t_diameter.append(di) 
    ba = np.pi*(di/2)**2
    t_basal_area.append(ba)  ## i.e. DBH in cm^2
    # this is biomass :
    # https://www.fao.org/3/w4095e/w4095e06.htm#3.2.1%20biomass%20regression%20equations 
    # using the dry equation (Y =10^{-0.535+log10 (BA)}) ---> this gives kg/tree
    #bio = 10**(-0.535 + np.log10(ba))
    bio = np.exp(-1.996+2.32*np.log(di))
    t_biomass.append(bio)
    #########################################################################


# adding new column to all_tree_data ##
## works !!! 
all_tree_data[tree_variables[0]] = t_diameter
all_tree_data[tree_variables[1]] = t_basal_area
all_tree_data[tree_variables[2]] = t_biomass

all_tree_data = all_tree_data.replace({0: None})

##  put tree data into csv file ################## so now all tree numbers are in 1 place #################
all_tree_data.to_csv(tree_csv_data,index=False, mode='w')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  now we want to compile the other bits to 1 csv  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
collated_data = pd.DataFrame(index = site_names)

all_tree_data = pd.read_csv(tree_csv_data)

# need to make individual lists for mean + group variables
tim = []
pre = []
alt = []
asp = []

bio = []
biol = []
lit = []
litl = []
prel = []
altl = []
ws = []
wsl = []
wd = []
wdl = []
rh = []
rhl = []
temp = []
templ = []
ne = []
nel = []
co = []
col = []

ne_l = []
ne_ll =[]
ne_w = []
ne_wl = []


lat = []
lon = []

nn = []
nnl = []
bd = []
bdl = []

print(site_section_data)

for j in range(n_sites):
    # add list of values to 
    tim.append(site_section_data[site_names[j]]['Time'][site_row_start[j]])
    lat.append(site_section_data[site_names[j]]['Lat'][site_row_start[j]])
    lon.append(site_section_data[site_names[j]]['Long'][site_row_start[j]])
    asp.append(site_section_data[site_names[j]]['aspect'][site_row_start[j]])

    #for t in range(site_row_end[j]-site_row_start[j]):
        #nn.append(site_section_data[site_names[j]]['nearest_neighbour'][site_row_start[j]+t])
        #bd.append(all_tree_data['biomass'][site_row_start[j]+t])
        
    #nn = np.mean(nn)
    #bd = np.sum(bd) 
    ############################# calculate nearest neighbour for each site:       #########################################
    #nnl.append((nn)/(0.5*np.sqrt(site_area/(site_row_end[j]-site_row_start[j]))))
    ############################## calculate biomass for each site:      #########################################
    #bdl.append(bd/site_area)

    #nn = []
    #bd = []
    # also in this made some if functions to make sure correct 
    for i in range(3):
        # this is the loop to get mean of 3 values
        wind = site_section_data[site_names[j]]['Wind_speed'][site_row_start[j]+i]
        if wind != None:
            print(wind)
            wind = wind * (1000/3600) # convert to m/s
            ws.append(wind) 
        if site_section_data[site_names[j]]['Wind_speed'][site_row_start[j]+i] != None:
            wd.append(site_section_data[site_names[j]]['Wind_direction'][site_row_start[j]+i])
        RH = site_section_data[site_names[j]]['RH'][site_row_start[j]+i]
        if RH != None:
            rh.append(RH)
        Temp = site_section_data[site_names[j]]['Temperature'][site_row_start[j]+i]
        if Temp != None:
            temp.append(Temp)
        pressure = site_section_data[site_names[j]]['Pressure'][site_row_start[j]+i]
        if pressure != None:
            pre.append(pressure * 100) # convert to pa
        altitude = site_section_data[site_names[j]]['altitude'][site_row_start[j]+i]
        if altitude != None:
            alt.append(altitude)

    prel.append(np.mean(pre))
    altl.append(np.mean(alt))
    wsl.append(np.mean(ws))
    wdl.append(np.mean(wd))
    rhl.append(np.mean(rh))
    templ.append(np.mean(temp))
    ws = []
    wd = []
    rh = []
    temp = []
    pre = []
    alt =[]
    
    ## this loop goes through all site section data and 
    for p in range(site_row_end[j]-site_row_start[j]):
        #print(site_row_end[j]-site_row_start[j])
        if site_section_data[site_names[j]]['cone'][site_row_start[j]+p] != None:
            cone = site_section_data[site_names[j]]['cone'][site_row_start[j]+p]
            co.append(cone * 10) #convert to mm
        
        if site_section_data[site_names[j]]['litter'][site_row_start[j]+p] != None:
            litter = site_section_data[site_names[j]]['litter'][site_row_start[j]+p]
            #print(litter)
            lit.append(litter * 10)  # convert to mm 

        if all_tree_data['biomass'][site_row_start[j]+p] > 0:
            #print(all_tree_data['biomass'][site_row_start[j]+p])
            bio.append(all_tree_data['biomass'][site_row_start[j]+p])

        if site_section_data[site_names[j]]['needle_l'][site_row_start[j]+p] != None:
            ne_l.append(site_section_data[site_names[j]]['needle_l'][site_row_start[j]+p])
    
        if site_section_data[site_names[j]]['needle_w'][site_row_start[j]+p] != None:
            ne_w.append(site_section_data[site_names[j]]['needle_w'][site_row_start[j]+p])


    biol.append(np.mean(bio))
    col.append(np.mean(co))
    litl.append(np.mean(lit))
    ne_ll.append(np.mean(ne_l))
    ne_wl.append(np.mean(ne_w))
    co = []
    bio = []
    lit = []
    ne_l = []
    ne_w = []

collated_data.insert(0,'Time',tim,allow_duplicates = True)
collated_data.insert(1,'Pressure',prel,allow_duplicates = True)
collated_data.insert(2,'Altitude',altl,allow_duplicates = True)

#collated_data.insert(4,'nearest_neighbour',nnl,allow_duplicates = True)
#collated_data.insert(5,'needle',nel,allow_duplicates = True)

collated_data.insert(3,'wind_speed',wsl,allow_duplicates = True)
collated_data.insert(4,'wind_direction',wdl,allow_duplicates = True)
collated_data.insert(5,'RH',rhl,allow_duplicates = True)
collated_data.insert(6,'Temp',templ,allow_duplicates = True)

collated_data.insert(7,'cone',col,allow_duplicates = True)
collated_data.insert(8,'aspect',asp,allow_duplicates = True)
collated_data.insert(9,'litter',litl,allow_duplicates = True)
collated_data.insert(10,'biomass',biol,allow_duplicates = True)

collated_data.insert(11,'needle_l',ne_ll,allow_duplicates = True)
collated_data.insert(12,'needle_w',ne_wl,allow_duplicates = True)


collated_data.insert(13,'Lat',lat,allow_duplicates = True)
collated_data.insert(14,'Long',lon,allow_duplicates = True)

collated_data.to_csv(collated_transect_csv, mode='w')


[5, 15, 22, 32, 40, 50, 56, 62]
{'site-1':    index tree_circumference litter altitude        Lat       Long Wind_speed  \
0      0               0.31    5.0   2148.0  28.745547 -17.831117        2.0   
1      1               1.97   17.0   2148.0       None       None        1.4   
2      2               0.05    0.0   2148.0       None       None        1.8   
3      3               None   None     None       None       None       None   
4      4               None   None     None       None       None       None   

  Wind_direction    RH Temperature Pressure   Time  cone aspect needle_l  \
0          147.0  72.3        10.5    795.8  12:04  13.0  182.0     22.0   
1          149.0  64.8         9.9    795.8   None  13.0   None     20.0   
2          138.0  69.7         7.4    795.8   None  10.0   None     20.0   
3           None  None        None     None   None   8.0   None     21.0   
4           None  None        None     None   None   9.0   None     22.0   

  needle_w  
0     

C:\Users\ezrie\AppData\Local\Temp\ipykernel_9608\3439865089.py:147: RuntimeWarning: divide by zero encountered in log
  bio = np.exp(-1.996+2.32*np.log(di))


4.9
5.3
5.9
2.7
2.6
3.2
5.2
6.2
4.6
0.0
0.0
0.0
1.4
1.4
0.0


PermissionError: [Errno 13] Permission denied: 'transect_1_processed.csv'

In [4]:
import numpy as np
n = [9,0,4,6,7,0,2,0,0,0]
print(np.mean(n))
print(np.mean(np.nonzero(n)))


2.8
3.0
